In [ ]:
import numpy as np


In [ ]:
###############第一节课：搭建网络框架、初始化网络参数##########################
dimensions=[28*28, 10]    #第一个参数是图片的维度，第二个参数是output的维度

In [ ]:
#定义激活函数
def tanh(x):
    x=np.array(x)
    return np.tanh(x)

    
def softmax(x):
    x=np.array(x)
    exp = np.exp(x)   # x-x.max()的作用是防止指数爆炸，在经过相减运算后，不影响最后的结果
    return exp/exp.sum()      #因为在代码中使用了x.max()，而普通数组是没有这样的max函数的，而np.array()有
                              #所以在传参进softmax(x)时，要传进来np.array(x)

In [ ]:
#定义初始化参数的函数
def init_parematers(input_dim, output_dim):  #参数为输入图片的维度和输出图片的维度
    b0 = np.zeros(input_dim)   #b0为全零，大小为784*1
    b1 = np.zeros(output_dim)  #b1为全零，大小为10*1
    w1 = np.random.rand(input_dim, output_dim)  #w1为随机，大小为784*10
    
    init_parameters_b_w = [   #以字典的形式返回所有参数
        {'b0':b0},
        {'b1':b1, 'w1':w1}
    ]
    return init_parameters_b_w


In [ ]:
##########################第二节课：读取并显示MNIST中的数据##################################
#设置训练集，验证集和测试集
train_set_num = 50000
valid_set_num = 10000
test_set_num = 10000

In [ ]:
#读取MNIST文件
import os

root_path = os.path.abspath('.')  #获取当前文件夹的绝对路径

train_img_path = os.path.join(root_path,"MNIST\\train-images.idx3-ubyte") #join()函数能将两个参数地址拼接到一起
train_lab_path = os.path.join(root_path,"MNIST\\train-labels.idx1-ubyte")

test_img_path = os.path.join(root_path,"MNIST\\t10k-images.idx3-ubyte")
test_lab_path = os.path.join(root_path,"MNIST\\t10k-labels.idx1-ubyte")

# 读取训练集中的图片数据
with open(train_img_path, 'rb') as t:  
    t.seek(16)    #因为train-images.idx3-ubyte中，前16个字节的数据是描述这个文件的，所以真正的数据是从16后面开始的
    train_img = np.fromfile(t,dtype=np.uint8).reshape(-1,28*28)  #np.fromfile()作用：从文本或二进制文件中的数据构造一个数组
    #reshape(-1,28*28)的解释：-1代表自动检测，因为不知道源文件中有多少个手写体数字，但是知道每个图片都是28*28的，所以第一个参数为-1就代表自动检测文件中有多少个28*28的图片
    train_img = train_img[:train_set_num]   #从60000训练集中分离出前50000个为真正的训练集
    valid_img = train_img[valid_set_num:]   #从60000训练集中分离出后10000个为验证集

# 读取训练集中的标签数据
with open(train_lab_path, 'rb') as t:  
    t.seek(8)    #在train-labels.idx1-ubyte中，前8个字节的数据是描述这个文件的
    train_lab = np.fromfile(t,dtype=np.uint8)
    train_lab = train_lab[:train_set_num]   
    valid_lab = train_lab[valid_set_num:]

# 读取测试集中的图片数据
with open(test_img_path, 'rb') as t:  
    t.seek(16) 
    test_img = np.fromfile(t,dtype=np.uint8).reshape(-1,28*28)


# 读取测试集中的标签数据
with open(test_lab_path, 'rb') as t:  
    t.seek(8)    
    test_lab = np.fromfile(t,dtype=np.uint8)
    

In [ ]:
#把数据画出来
import matplotlib.pyplot as plt

#画出训练集数据
def show_train_img(index):
    img = train_img[index].reshape(28,28)  #前面读取图片时的28*28是告诉程序每一张图片有多大，最后仍然是按行来储存读出来的数据（即:1*784的一维数据），而此处的28*28则是将读出来的那个一维数据转换成二维交给画图程序画图
    plt.imshow(img, cmap="gray")     #用灰度的格式将图片画出来
    print("train_data:{}".format(train_lab[index]))     #打印图片对应的标签

#画出验证集数据
def show_valid_img(index):
    img = valid_img[index].reshape(28,28)
    plt.imshow(img,cmap="gray")
    print("valid_data:{}".format(valid_lab[index]))

#画出测试集数据
def show_test_img(index):
    img = test_img[index].reshape(28,28)
    plt.imshow(img,cmap="gray")
    print("test_data:{}".format(test_lab[index]))

In [ ]:
#显示训练集图片和标签
show_train_img(np.random.randint(60000))

In [ ]:
#显示验证集的图片和标签
show_valid_img(np.random.randint(10000))

In [ ]:
#显示测试集图片和标签
show_test_img(np.random.randint(10000))

In [ ]:
################第四节课：反向传播和梯度下降更新网络中的参数#################################
# 计算梯度下降所用到的导数
# 注意！！！(传参时，参数中所有的x必须是np的数组对象)
h=0.0001

#用导数极限的定义求tanh的导数
def d_tanh_approximation(x):   
    x=np.array(x)
    return ((tanh(x+h)-tanh(x))/h)

#直接对tanh求导，得到准确的值
def d_tanh_precise(x):   
    x=np.array(x)
    return 1-tanh(x)**2

#用导数极限的定义求softmax的导数
def d_softmax(x):
    x=np.array(x)
    return ((softmax(x+h)-softmax(x))/h)


In [ ]:
#计算y_real和y_predict的差值
def diff_yreal_ypredict(lab_num,flatten_img):
    b0=init_parematers(784, 10)[0]['b0']
    w1=init_parematers(784, 10)[1]['w1']
    b1=init_parematers(784, 10)[1]['b1']

    l0_in=flatten_img+b0  #l0=tanh(data+b0)
    l0_out=tanh(l0_in)

    l1_in=np.dot(l0_out,w1)+b1
    l1_out=softmax(l1_in)

    y_predict=l1_out #用y_predict来代表预测值

    y_real=[0 for i in range(10)]  #y_real为长度为10的数组
    y_real=np.eye(10)              #生成10*10的对角矩阵
    y_real=y_real.reshape(-1,1*10)  #用y_real来代表真实世界的值，y_real[0]就代表[1 0 0 0 0 0 0 0 0]，即label=1的数字的理想输出, y_real[1]同理
    
    return y_real[lab_num]-y_predict  #返回差值

In [ ]:
#计算梯度下降

#求L对b1的偏导数=-2 * (y-output) * d_A2
# 注意！！！(lab_num是从0开始的，所以lab_num=1代表标签2)
def L_to_b1(lab_num,flatten_img):
    b0=init_parematers(784, 10)[0]['b0']
    w1=init_parematers(784, 10)[1]['w1']
    b1=init_parematers(784, 10)[1]['b1']

    l0_in=flatten_img+b0  #l0=tanh(data+b0)
    l0_out=tanh(l0_in)
    
    l1_in=np.dot(l0_out,w1)+b1

    d_A2=d_softmax(l1_in)  #求softmax的偏导

    diff=diff_yreal_ypredict(lab_num-1,flatten_img)  #求y-output的差值

    result=-2*(d_A2*diff)    #计算L对b1的偏导数的结果
   
    return result

#求L对w1的偏导数=-2 * (y-output) * d_A2 * l0
# 注意！！！(lab_num是从0开始的，所以lab_num=1代表标签2)
def L_to_w1(lab_num,flatten_img):
    b0=init_parematers(784, 10)[0]['b0']
    w1=init_parematers(784, 10)[1]['w1']
    b1=init_parematers(784, 10)[1]['b1']

    l0_in=flatten_img+b0  #l0=tanh(data+b0)
    l0_out=tanh(l0_in)
    
    l1_in=np.dot(l0_out,w1)+b1

    d_A2=d_softmax(l1_in)  #求softmax的偏导

    diff=diff_yreal_ypredict(lab_num-1,flatten_img)  #求y-output的差值

    mid_temp=d_A2*diff    #计算L对b1的偏导数的结果

    l0_out=l0_out.reshape(784,1)
    mid_temp=mid_temp.reshape(1,10)

    result=np.dot(l0_out,mid_temp)
    
    return result


#求L对b1的偏导数=-2 * (y-output) * d_A2 * w1 * d_A1  (d_A2代表队A2求导，A2是第二层的激活函数“softmax”，A1是第一层的激活函数“tanh”)
# 注意！！！(lab_num是从0开始的，所以lab_num=1代表标签2)
def L_to_b0(lab_num,flatten_img):
    b0=init_parematers(784, 10)[0]['b0']
    w1=init_parematers(784, 10)[1]['w1']
    b1=init_parematers(784, 10)[1]['b1']

    l0_in=flatten_img+b0  #l0=tanh(data+b0)
    l0_out=tanh(l0_in)
    
    l1_in=np.dot(l0_out,w1)+b1  #l1=softmax(w1*l0+b1)   (注意！)内积运算中，l0和w1的位置不能颠倒

    d_A1=d_tanh_precise(l0_in)  #求tanh的偏导
    d_A2=d_softmax(l1_in)  #求softmax的偏导

    diff=diff_yreal_ypredict(lab_num-1,flatten_img)  #求：y-output

    mid_temp_1=diff*d_A2   #求：(y-output)*d_A2

    mid_temp_2=np.dot(w1,mid_temp_1)   #求：w1*(y-output)*d_A2

    result = mid_temp_2*d_A1   #求：w1*(y-output)*d_A2*d_A1

    return result
